In [ ]:
import pandas as pd

In [ ]:
# importing the dataset
df = pd.read_csv('/content/SMSSpamCollection.txt',sep = '\t',names = ['label','message'])

In [ ]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df['label'].unique()

array(['ham', 'spam'], dtype=object)

In [ ]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
# converting columns to a list
x = list(df['message'])
y = list(df['label'])

In [ ]:
y=list(pd.get_dummies(y,drop_first=True)['spam'])

In [ ]:
# train and test data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 4)

In [ ]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(x_train,truncation=True,padding=True)
test_encodings = tokenizer(x_test,truncation=True,padding=True)

In [ ]:
# converting the encodings into dataset objects
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings),y_test))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_steps = 500
)

In [ ]:
# training the model

with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
trainer.train()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [ ]:
# evaluating the loss
trainer.evaluate(test_dataset)

{'eval_loss': 0.0384096554347447}

In [ ]:
trainer.predict(test_dataset)

PredictionOutput(predictions=array([[ 3.7754028, -3.5618062],
       [ 3.8021286, -3.5804112],
       [ 3.6202579, -3.426295 ],
       ...,
       [ 3.771827 , -3.5708904],
       [ 3.7229564, -3.5550098],
       [-3.1238813,  3.2123165]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 1], dtype=int32), metrics={'eval_loss': 0.038749170303344724})

In [ ]:
# save the model
save_location = "trained_model"

model.save_pretrained(save_location)
tokenizer.save_pretrained(save_location)

('trained_model/tokenizer_config.json',
 'trained_model/special_tokens_map.json',
 'trained_model/vocab.txt',
 'trained_model/added_tokens.json')

In [ ]:
# load the model

tokenizer_trained_model = DistilBertTokenizer.from_pretrained(save_location)
trained_model = TFDistilBertForSequenceClassification.from_pretrained(save_location)

Some layers from the model checkpoint at trained_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at trained_model and are newly initialized: ['dropout_99']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Prediction

import numpy as np

def predictive_system(input_text):

  ans_class = ['Not Spam','Spam']

  predict_output = tokenizer_trained_model.encode(input_text, truncation=True , padding = True )

  output = trained_model.predict(predict_output)[0]

  return ans_class[np.argmax(output)]

In [ ]:
print('predicted output',predictive_system(x_test[3]))
print('actual output',y_test[3])

1/1 [==============================] - 0s 59ms/step
predicted output Not Spam
actual output 0
